In [ ]:
import numpy as np
import pandas as pd
from scipy.io.wavfile import read 
import os

In [ ]:
# Storing the address in 'url' variable
data_path =   "dsl_data/development.csv"
# Importing the file from the address contained in 'url' into 'df' 
df = pd.read_csv(data_path)

In [ ]:
# Check null values in Training dataset (pandas dataframe)
df.isnull().sum().sum()

In [ ]:
# To get unique values in order to map values to objects imported from src/data.py

print(df['object'].unique())
print(df['action'].unique())


In [ ]:
df.shape

In [ ]:
import dict_data 

df['gender'] = df['gender'].map(dict_data.gender_map)
df['Self-reported fluency level '] = df['Self-reported fluency level '].map(dict_data.language_fluency_map)
df['ageRange'] = df['ageRange'].map(dict_data.age_range_map)
df['Current language used for work/school'] = df['Current language used for work/school'].map(dict_data.current_language_map)
df['First Language spoken'] = df['First Language spoken'].map(dict_data.first_language_map)


# Try to combine action & object in 1 column
df["action-object"] = df['action'].astype(str) +"-"+ df["object"]


In [ ]:
import librosa

audios_array = []
duration_array = []
zero_crossing_array = []
mfcc_array = []


for audio in df['path']:
    # rate, data = read(audio, mmap=False) 

    # Load audio files (wav) as amplitude and rate using 
    # Default rate: 22050
    data ,rate = librosa.load(audio)

    zero_crossing_array.append(sum(librosa.zero_crossings(data)))
    audios_array.append(data)

    # feature_vector = get_feature_vector(data, rate)
    # norm_audios_feat.append(feature_vector) 

    mfcc = librosa.feature.mfcc(y=data, sr=rate)
    mfcc_array.append(mfcc) 

    # Get duration by dividing number of columns in data by rate (Number of channels)
    duration_array.append(data.shape[0] / rate)


df['zero_crossing'] = zero_crossing_array
df['audio'] = audios_array 
df['duration'] = duration_array 
df['mfcc'] = mfcc_array 



# we can realize two different sample rates for our wav files 16000 & 22050
# print(df.rate.value_counts())
# print(df.rate.unique())



In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split

x = df[['Self-reported fluency level ','First Language spoken', 'Current language used for work/school', 'gender', 'ageRange','zero_crossing','duration','mfcc']].copy()
y = df[['action-object']].copy()


#train_test_split splits the data into 75% training data and 25% test data
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [ ]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,y_train.values.ravel())
y_pred=clf.predict(x_test)

y_pred

In [ ]:
from sklearn import svm

clf = svm.SVC()
clf.fit(x_train,y_train)
y_pred_svm=clf.predict(x_test)
y_pred_svm


In [ ]:

# from sklearn.tree import DecisionTreeClassifier
# clf = DecisionTreeClassifier(max_depth = 10, min_impurity_decrease=0.01)
# clf.fit(x_train, y_train)
# y_pred = clf.predict(x_test)
# y_pred

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy using test data (25%)
print("Test set accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
# Storing the address in 'url' variable
evaluation_path =   "dsl_data/evaluation.csv"

# Importing the csv file from the address contained in 'url' into 'evaluation_df' 
evaluation_df = pd.read_csv(evaluation_path)

# Check null values in Training dataset (pandas dataframe)
evaluation_df.isnull().sum().sum()

import dict_data 

# map features in evaluation_df to predefined dictionaries
evaluation_df['gender'] = evaluation_df['gender'].map(dict_data.gender_map)
evaluation_df['Self-reported fluency level '] = evaluation_df['Self-reported fluency level '].map(dict_data.language_fluency_map)
evaluation_df['ageRange'] = evaluation_df['ageRange'].map(dict_data.age_range_map)
evaluation_df['Current language used for work/school'] = evaluation_df['Current language used for work/school'].map(dict_data.current_language_map)
evaluation_df['First Language spoken'] = evaluation_df['First Language spoken'].map(dict_data.first_language_map)


# Get audio file for each instance into an array
path = os.getcwd() + "/"
audios_array = []
rate_array = []

for audio in evaluation_df['path']:
    rate, data = read(audio, mmap=False)   
    data = np.mean(data) 
    audios_array.append(data)
    rate_array.append(rate)


# Save array as a new column in our evaluation dataframe
evaluation_df['audio'] = audios_array
evaluation_df['rate'] = rate_array


x_evaluation = evaluation_df[['Self-reported fluency level ','First Language spoken', 'Current language used for work/school', 'gender', 'ageRange', 'zero_crossing','duration','mfcc']].copy()



In [ ]:
y_evaluation=clf.predict(x_evaluation)

for element in y_evaluation:
    element = element.replace("none", "")
    element = element.replace("-", "")

remove_dash = list(map(lambda s: s.replace("-", ""), y_evaluation))
modified_y_evaluation = list(map(lambda s: s.replace("none", ""), remove_dash))

# print map(lambda s: s.replace('-' , 'n'), y_evaluation)



y_evaluation_df = pd.DataFrame(modified_y_evaluation, columns = ['Predicted'])
y_evaluation_df
y_evaluation_df.index.name = 'Id'

y_evaluation_df.to_csv('evaluation/predictions.csv',index=True,header=True)